In [ ]:
!pip install -U spacy -q

In [ ]:
!python -m spacy info


============================== Info about spaCy ==============================

spaCy version    3.3.0                         
Location         /usr/local/lib/python3.7/dist-packages/spacy
Platform         Linux-5.4.188+-x86_64-with-Ubuntu-18.04-bionic
Python version   3.7.13                        
Pipelines        en_core_web_sm (3.3.0)        



In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en") # load a new spacy model
db = DocBin() # create a DocBin object

In [ ]:
import json #importing train data 
f = open('/content/drive/MyDrive/Annotated_Data_2.json')
TRAIN_DATA = json.load(f)

In [ ]:
for text, annot in tqdm(TRAIN_DATA['annotations']): #converting to docbin file for training purpose
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents 
    db.add(doc)

db.to_disk("./training_data.spacy") # save the docbin object

100%|██████████| 49/49 [00:00<00:00, 81.07it/s]


In [ ]:
! python -m spacy init config config.cfg --lang en --pipeline ner --optimize accuracy #generating cofig file 

⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: accuracy
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
!python -m spacy download "en_core_web_lg" #vectors for accuracy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 400.7 MB 5.3 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:
! python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy #training model

ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2022-05-31 18:33:49,750] [INFO] Set up nlp object from config
[2022-05-31 18:33:49,766] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-05-31 18:33:49,771] [INFO] Created vocabulary
[2022-05-31 18:33:52,395] [INFO] Added vectors: en_core_web_lg
[2022-05-31 18:33:54,288] [INFO] Finished initializing nlp object
[2022-05-31 18:34:10,065] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00    259.81    0.00    0.00    0.00    0.00
  4     200       2282.94  14773.88   39.94   39.70   40.18    0.40
  8     400       1239.73   7448.36   68.32   65.07  

In [ ]:
nlp_ner = spacy.load("/content/model-best")

In [ ]:
doc = nlp_ner("FIROZ KHAN firozkhan089@gmail.com +91-9758565672 Career Objective: If you want to leave your footprints on the sands of time do not drag your feet- Dr. APJ Abdul Kalam Apropos, I will leave no stone unturned in making full utilization of my knowledge and talent to the utmost satisfaction of the organization I will work for and to myself Academic Qualifications: Degree University/Board Year Percentage Bachelor of Technology [MECHANICAL ENGINEERING] Uttar Pradesh Technical University 2014 64.6% Intermediate CBSE BOARD 2009 65.8% High School ICSE BOARD 2007 75% Area of Interest: Production Technology Automobile engineering Skill Qualifications : Basic ‘C’ language . Working on windows xp , windows 8. SUMMER TRAINING : One Month summer training at 510 ARMY BASE WORKSHOP at Meerut Cantt. Workshop Attended: Workshop on Advances in HVAC Industry . Personal Details: Father’s Name Mr. Rahat Ali Mother’s Name Mrs. Chaman Bano Languages Known English, Hindi Home Town Meerut Religion Muslim Marital Status Unmarried Height 173cms Date Of Birth 12th August 1991 Corresponding Address 12/128,sec-12 Shastri Nagar ,Meerut Email ID firozkhan089@gmail.com Contact number +91-9758565672 DATE : PLACE : Meerut Firoz Khan")

In [ ]:
spacy.displacy.render(doc, style="ent", jupyter=True)

In [36]:
import json
f = open('/content/testt.json')
TEST_DATA = json.load(f)

In [37]:
for text, annot in tqdm(TEST_DATA['annotations']): 
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents 
    db.add(doc)

db.to_disk("./test_data.spacy") # save the docbin object

100%|██████████| 1/1 [00:00<00:00, 81.88it/s]


In [42]:
!python -m spacy evaluate ./model-best/ ./test_data.spacy #evaluating the model

ℹ Using CPU

================================== Results ==================================

TOK     100.00
NER P   98.66 
NER R   98.29 
NER F   98.47 
SPEED   2806  


=============================== NER (per type) ===============================

                           P        R        F
OBJECTIVE             100.00   100.00   100.00
NAME                  100.00   100.00   100.00
DESIGNATION            98.00    99.32    98.66
CONTACT NUMBER        100.00    97.87    98.92
EMAIL ADDRESS         100.00   100.00   100.00
SKILLS                 98.81    98.42    98.62
DEGREE                 95.06    98.72    96.86
INSTITUTE              98.81    95.40    97.08
COMPANIES WORKED AT    99.05    98.11    98.58
LOCATION              100.00    88.24    93.75



In [47]:
#Cheking resume match with job description
from gensim.summarization.summarizer import summarize
from gensim.summarization import keywords# Import the library

In [43]:
resume_data = []
job_desc = []
with open('/content/drive/MyDrive/t_data.txt','r') as f:
  for line in f:
    resume_data.append(line)
f.close()
with open('/content/drive/MyDrive/job_data.txt','r') as f1:
  for line in f1:
    job_desc.append(line)
f.close()

In [51]:
actual_result = [3,2,3,4,4,3,3,2]
for i in range(0,8):
  k=resume_data[i]
  j= job_desc[i]
  try:
    summarize(k, ratio=0.1)
    text_list = [k, j]
    from sklearn.feature_extraction.text import CountVectorizer
    cv = CountVectorizer()
    count_matrix = cv.fit_transform(text_list)
    from sklearn.metrics.pairwise import cosine_similarity
    # get the match percentage
    matchPercentage = cosine_similarity(count_matrix)[0][1] * 100
    matchPercentage = round(matchPercentage, 2) # round to two decimal
    print('Your resume matches about '+ str(matchPercentage)+ '% of the job description.' ' Actual Labelled Match: ',actual_result[i])
  except:
    print("Can't")

Your resume matches about 47.93% of the job description. Actual Labelled Match:  3
Your resume matches about 38.64% of the job description. Actual Labelled Match:  2
Your resume matches about 55.47% of the job description. Actual Labelled Match:  3
Your resume matches about 42.53% of the job description. Actual Labelled Match:  4
Your resume matches about 53.32% of the job description. Actual Labelled Match:  4
Your resume matches about 42.75% of the job description. Actual Labelled Match:  3
Your resume matches about 53.48% of the job description. Actual Labelled Match:  3
Your resume matches about 43.72% of the job description. Actual Labelled Match:  2
